In [26]:
# Import the function to get connect to the db
from snowexsql.db import get_db
from snowexsql.data import SiteData, PointData, LayerData, ImageData
# This is what you will use for all of hackweek to access the db
db_name = 'snow:hackweek@db.snowexdata.org/snowex'

# This is what you will use for all of hackweek to access the db
engine, session = get_db(db_name)

In [27]:
# Start with the basic query
qry = session.query(LayerData.doi)

In [24]:
from datetime import datetime

# Filter for Grand Mesa, the specific date range, and density profiles
qry = qry.filter(
    LayerData.site_name == "Grand Mesa",
    LayerData.date >= datetime(2019, 12, 1),
    LayerData.date <= datetime(2020, 5, 31),
    LayerData.type == 'lwc_vol'
)

In [25]:
# Get distinct DOIs only
qry = qry.distinct()

# Optional: Protect ourselves from excessive data with a limit
qry = qry.limit(1000000000)

# Execute the query and fetch all results
result = qry.all()

# Extract the DOIs from the results
unique_dois = [r[0] for r in result]
print('Unique DOIs:', unique_dois)

session.close()

Unique DOIs: ['lwc_vol']


In [23]:
from datetime import datetime

# This is what you will use for all of hackweek to access the db
engine, session = get_db(db_name)

# Query to get all unique types
qry = session.query(LayerData.type).filter(
    LayerData.site_name == "Grand Mesa",
    LayerData.date >= datetime(2019, 12, 1),
    LayerData.date <= datetime(2020, 5, 31)
).distinct()

available_types = qry.all()

session.close()

# Print available types
print('Available Types:', [r[0] for r in available_types])


Available Types: ['sample_signal', 'force', 'density', 'grain_size', 'reflectance', 'permittivity', 'lwc_vol', 'manual_wetness', 'equivalent_diameter', 'specific_surface_area', 'grain_type', 'temperature', 'hand_hardness']


In [32]:
# Reflect the tables in the database
from sqlalchemy import inspect

# Create an inspector to view the schema
inspector = inspect(engine)

# Get a list of all tables in the database
tables = inspector.get_table_names()

# Print the list of tables
print("Available tables:", tables)


Available tables: ['spatial_ref_sys', 'points', 'layers', 'sites', 'images']


In [34]:
from sqlalchemy import text

# Form the SQL query to get distinct DOIs from the layers table
qry = """
SELECT DISTINCT doi
FROM layers
WHERE site_name = 'Grand Mesa'
AND date >= '2019-12-01'
AND date <= '2020-05-31'
AND type = 'density'
"""

# Establish a connection and execute the query
with engine.connect() as connection:
    results = connection.execute(text(qry))

    # Create a readable string to print the DOIs
    # Access tuple values by index (0) since the results are tuples
    out = ', '.join(row[0] for row in results)

    # Print the distinct DOIs with a line return for readability
    print(out + '\n')


https://doi.org/10.5067/DUD2VZEVBJ7S, https://doi.org/10.5067/KZ43HVLZV6G4

